In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Libraries**

In [30]:
import cuml
import cudf as pd
import cupy as cp
from cuml import PCA
from cuml.decomposition import PCA
from cuml.linear_model import LinearRegression
from cuml.model_selection import train_test_split
import seaborn as sns

**Loading Data**

In [31]:
train_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

**Data Preprocessing**

In [32]:
numerical_features = train_df.dtypes[train_df.dtypes != "object"].index
print("No. of numerical features  ", len(numerical_features))

In [33]:


categorical_features = train_df.dtypes[train_df.dtypes == "object"].index
print("No. of categorical features  ", len(categorical_features))

In [34]:
for a in list(categorical_features) :
    print(train_df[a].value_counts())
    print('////////////////////////////////////////////////////////2')

In [35]:
feature_list_mode = ['BsmtCond','BsmtQual','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','MasVnrType','MasVnrArea','BsmtExposure','BsmtFinType2','Utilities','Exterior1st','Exterior2nd','BsmtFinType1','SaleType','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional',]
feature_list_mean = ['GarageCars','GarageArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','LotFrontage']


In [36]:
train_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)
test_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)

In [37]:
for a in feature_list_mode:
    train_df[a] = train_df[a].fillna(train_df[a].mode()[0])
    


In [38]:
for a in feature_list_mean:
    train_df[a] = train_df[a].fillna(train_df[a].mean())

In [39]:
for a in feature_list_mode:
    test_df[a] = test_df[a].fillna(test_df[a].mode()[0])
    


In [40]:
for a in feature_list_mean:
    test_df[a] = test_df[a].fillna(test_df[a].mean())

In [41]:
train_df.dropna(inplace=True)

In [42]:
train_df.isnull().sum()

In [43]:
final_data = pd.concat([train_df, test_df],axis=0)

In [44]:
final_data1 = pd.get_dummies(final_data)

In [45]:
for column in final_data1.columns:
    if final_data1[column].dtype == 'O':
        final_data1.drop([column], axis=1, inplace=True)

In [46]:
Train=final_data1.iloc[:1459,:]
Test=final_data1.iloc[1459:,:]

In [47]:
X=Train.drop(['SalePrice'],axis=1)
Y=Train['SalePrice']
Test=Test.drop(['SalePrice'],axis=1)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.29)

In [49]:
linreg = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
split=linreg.fit(X_train, y_train)

In [50]:
print("R-Squared Value for Training Set:",(linreg.score(X_train.astype('float32'), y_train.astype('float32'))))

In [51]:
y_pred_split=linreg.predict(Test)
y_pred_split

In [52]:
print("Coefficients:")
print(split.coef_)
print("Intercept:")
print(split.intercept_)



In [53]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

In [54]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X,Y)
    preds = lr.predict(Test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

without train test split 

In [55]:
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
reg = lr.fit(X.values,Y)
y_pred=lr.predict(Test)
y_pred

In [56]:
print("R-Squared Value for Training Set:",lr.score(X.astype('float32'), Y.astype('float32')))

In [57]:
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)



In [58]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

In [59]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X,Y)
    preds = lr.predict(Test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred.astype('int64')))

In [60]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

In [61]:
sample_sub['SalePrice'] = y_pred_split
sample_sub.to_csv('submissionshrey.csv', index=False)